## Project 5
- created 5-16-22 by GTP
- https://docs.google.com/document/d/1LIJTlCsx54zIG5sOX3heSj00YqdzhZ9c/edit
- *Description*: BSEED has entered a lot of data into free text fields within Accela. Would be useful to find ways to scrape and organize this data so it is useable. Unit data and Certificates of Occupancy are some of our biggest gaps. This might be a way to use administrative data to version and validate 2020 data.
- Technical Skill Level: Medium-High. Skilled at applying Regex to text strings using SQL and/or Python. Experience working with geospatial data, in ArcGIS or otherwise.
- Scope: There are 595 records in the Certificates of Occupancy dataset and 5,930 records in the Certificates of Compliance dataset. Depending on skill level, this could take 6-8 weeks.
- Inputs: Certificates of Compliance, Certificates of Occupancy, Rental Registration data
- General Process:
- Use GIS or Base Units Explorer tool to link Certificates of Occupancy to specific building ids, to create timestamps for when a building was ready for occupants.
- Geocode the addresses in the Certificate of Compliance and Rental Registration datasets and note any addresses that can’t be matched through a manual rematching process and may be missing altogether from the database.


In [2]:
#import data libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numbers
import decimal
#import data science packages
import scipy
import scipy.stats as stats

np.random.seed(222)
%matplotlib inline

In [48]:
import re

In [3]:
#import geographic analysis libraries
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely as shp
from shapely.geometry import Point
from shapely.geometry import shape
import os
import re
from fiona.crs import from_epsg
import pysal as ps

/Users/geoffperrin/miniconda3/lib/python3.8/site-packages/pysal/explore/segregation/network/network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
/Users/geoffperrin/miniconda3/lib/python3.8/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [4]:
#set crs for entire analysis
crs = {'init': 'epsg:4326'}

### data sources

Certificates of Occupancy: https://data.detroitmi.gov/datasets/certificates-of-occupancy-1/explore
- BSEED says that new building (or a rehabbed / renovated building) has satisfied their requirements for habitation, and people can move in / it is ready for occupancy
- note: alice says that this can be issued for individual floors 
- _goal_: deliverable should be a table that is the certificate of occupancy number (record_id) and building footprint ids - sometimes these are 1 to 1 and then sometimes multiple occupancy numbers might relate to a single id
- the census challenge is interested in having this as a record to when exactly a new building was technically 'habitable' - the "birthdate" of the property in terms of occupancy

Certificates of Compliance: https://data.detroitmi.gov/datasets/certificates-of-compliance-1/explore
- this is for properties to be certified as 'compliant' by the city
- Alice has access to the dataset of compliance that has 'description' - which should contain additional details..?
- I think this will be more trying to geocode the ones that don't have lat/lon
- _goal_: there's 33 that didn't geocode - goal is to geocode these and then give description if couldn't geocode

Rental Registrations: https://data.detroitmi.gov/datasets/rental-statuses-1/explore
- (6-1-22): I'll address this next week with Alice on our next call

Base Units: https://base-units-detroitmi.hub.arcgis.com/datasets/detroitmi::units-1/about
- jimmy mcbroom put this together

In [5]:
compliance_gdf = gpd.read_file('../data/Certificates_Of_Compliance/Certificates_Of_Compliance.shp')

In [11]:
len(compliance_gdf)

6071

In [80]:
compliance_gdf.sample(5)

,record_id,street_num,street_dir,street_nam,street_typ,task,status,record_sta,parcel_id,lon,lat,ObjectId,geometry
693,PMB2004-10130,3733,None,RIVARD,None,Issue CofC,Issued,2021-06-11,05003663-708,-83.047632,42.353477,694,POINT (-83.04763 42.35348)
6023,PMB2022-01230,2623,W,MCNICHOLS UNIT 13/106,None,Issue CofC,Issued,2022-04-19,CONDO BUILDING,-83.128693,42.417075,6024,POINT (-83.12869 42.41707)
2289,PMB2012-04553,14596,None,PIERSON,None,Issue CofC,Issued,2020-03-02,22106450-1,-83.245426,42.394771,2290,POINT (-83.24543 42.39477)
1363,PMB2005-11377,15834,None,WESTBROOK,None,Issue CofC,Issued,2019-07-29,22109784.,-83.250414,42.406407,1364,POINT (-83.25041 42.40641)
4410,PMB2018-06938,17625,None,PATTON,None,Issue CofC,Issued,2019-11-15,22104390.,-83.245354,42.420823,4411,POINT (-83.24535 42.42082)


In [21]:
len(compliance_gdf[compliance_gdf['geometry'].isna()])

33

In [62]:
compliance_gdf[compliance_gdf['geometry'].isna()]

,record_id,street_num,street_dir,street_nam,street_typ,task,status,record_sta,parcel_id,lon,lat,ObjectId,geometry
40,PMB2003-01577,924,E,LAFAYETTE,None,Issue CofC,Issued,2021-08-24,None,NaN,NaN,41,None
55,PMB2003-01845,2671,None,LAFAYETTE,None,Issue CofC,Issued,2021-06-09,None,NaN,NaN,56,None
141,PMB2004-03507,9658,None,NORTHLAWN,None,Issue CofC,Issued,2019-11-01,None,NaN,NaN,142,None
165,PMB2004-04234,287,None,EDSEL FORD,None,Issue CofC,Issued,2021-03-08,None,NaN,NaN,166,None
260,PMB2004-14249,1387,None,LARNED,None,Issue CofC,Issued,2021-05-19,None,NaN,NaN,261,None
831,PMB2004-10797,4727,None,THIRD,None,Issue CofC,Issued,2020-01-30,None,NaN,NaN,832,None
833,PMB2004-10802,930,W,FOREST,None,Issue CofC,Issued,2020-01-30,None,NaN,NaN,834,None
843,PMB2004-10938,13401,E,SEVEN MILE,Rd,Issue CofC,Issued,2019-01-11,None,NaN,NaN,844,None
1109,PMB2005-14865,1330,None,PLUM,None,Issue CofC,Issued,2021-06-05,None,NaN,NaN,1110,None
1140,PMB2005-19368,1511,E,LARNED,None,Issue CofC,Issued,2021-07-23,None,NaN,NaN,1141,None


In [24]:
compliance_gdf.sample(2)

,record_id,street_num,street_dir,street_nam,street_typ,task,status,record_sta,parcel_id,lon,lat,ObjectId,geometry
4020,PMB2018-01852,14101,None,PIEDMONT,None,Issue CofC,Issued,2019-10-23,22087642.,-83.230062,42.388963,4021,POINT (-83.23006 42.38896)
1196,PMB2005-16627,2230,None,CHENE,None,Issue CofC,Issued,2021-11-24,11002894-915,-83.028616,42.348217,1197,POINT (-83.02862 42.34822)


In [7]:
occupancy_gdf = gpd.read_file('../data/Certificates_Of_Occupancy/Certificates_Of_Occupancy.shp')

In [17]:
len(occupancy_gdf[occupancy_gdf['geometry'].isna()])/len(occupancy_gdf)

0.16166666666666665

In [26]:
len(occupancy_gdf[occupancy_gdf['geometry'].isna()])

97

In [63]:
occupancy_gdf[occupancy_gdf['geometry'].isna()]

,record_id,street_num,street_dir,street_nam,street_typ,descriptio,status,date_statu,parcel_id,lon,lat,ObjectId,geometry
4,BLD2017-06254,8401,None,WOODMONT 14,None,"SEE BLD2017-00831\nAKA 8227, 8237, 8243, 8251 ...",CofO Issued,2019-02-23,None,NaN,NaN,5,None
12,BLD2019-03478,1541,None,Fisher Frwy,None,AKA 1541 W. Fisher Freeway Unit 29. Per BZA #4...,CofO Issued,2020-08-12,None,NaN,NaN,13,None
23,BLD2019-04976,2327,None,Trumbull,None,"AKA 2327 Trumbull Ave. Unit 20. Per BZA #4-18,...",CofO Issued,2020-10-15,None,NaN,NaN,24,None
25,BLD2018-05987,1230,None,LIBRARY,None,PERMANENT CERTIFICATE OF OCCUPANCY ISSUED,CofO Issued,2019-04-22,None,NaN,NaN,26,None
36,BLD2019-00054,2817,None,BRUSH,None,"ERECT A 4 STORY, TOWNHOME AS PER EPLANS AND CE...",CofO Issued,2021-06-01,None,NaN,NaN,37,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,BLD2017-09621,286,None,ALFRED,None,"(AKA 292 ALFRED) ERECT A 3 STORY, TOWNHOUSE W/...",CofO Issued,2021-09-07,None,NaN,NaN,577,None
582,BLD2019-00679,2807,None,Brush,None,New residential structure 8 units 4 story tow...,CofO Issued,2021-05-28,None,NaN,NaN,583,None
592,BLD2020-00477,4501,None,St. Aubin,None,Revision to BLD2019-02421 per plans.\r\n(Const...,CofO Issued,2021-02-24,None,NaN,NaN,593,None
595,BLD2019-02529,692,None,AMSTERDAM,None,Revision to BLD2018-07772 to reflect changes t...,CofO Issued,2020-08-19,None,NaN,NaN,596,None


In [71]:
occupancy_gdf['descriptio'][occupancy_gdf['record_id']=='BLD2021-02415'].values[0]

'(AKA 3321 Cochrane) Construct (11) unit Rowhouse building and Accessory Garages per BZA (41-19) & (SLU2019-00020) per Plans.\r\n(Permit reviewed under BLD2019-03775)'

In [72]:
occupancy_gdf[occupancy_gdf['record_id']=='BLD2021-02415']

,record_id,street_num,street_dir,street_nam,street_typ,descriptio,status,date_statu,parcel_id,lon,lat,ObjectId,geometry
467,BLD2021-02415,3303,None,COCHRANE,None,(AKA 3321 Cochrane) Construct (11) unit Rowhou...,CofO Issued,2021-10-11,08006537.001,-83.074239,42.339555,468,POINT (-83.07424 42.33956)


In [12]:
len(occupancy_gdf)

600

## Occupancy DF
- notes: descriptio column has free text that we could leverage to fill in empty geometry cells
- "AKA 2327 Trumbull Ave" is an example of the free text - as in, AKA "address" is a common pattern, but this already exists / has been extracted into the street_num / street_nam columns - maybe those just have to be geocoded

- we're looking for the relationship between housing units and certificates of occupancy. That relationship is often mediated by a building id (use https://cityofdetroit.github.io/base-unit-tools/explorer?id=3263&type=buildings&streetview=true) / there's a xwalk that alice will send over

- deliverable should be a table that is the certificate of occupancy number (record_id) and building footprint ids - sometimes these are 1 to 1 and then sometimes multiple occupancy numbers might relate to a single id

- the census challenge is interested in having this as a record to when exactly a new building was technically 'habitable' - the "birthdate" of the property in terms of occupancy

In [27]:
occupancy_gdf_empty = occupancy_gdf[occupancy_gdf['geometry'].isna()]

In [79]:
occupancy_gdf[~occupancy_gdf['parcel_id'].isna()].sample(10)

,record_id,street_num,street_dir,street_nam,street_typ,descriptio,status,date_statu,parcel_id,lon,lat,ObjectId,geometry
199,BLD2017-04839,2645,None,WOODWARD,None,PERMANENT CERTIFICATE OF OCCUPANCY ISSUED\nREV...,CofO Issued,2019-04-02,02000572-638,-83.054996,42.341031,200,POINT (-83.05500 42.34103)
589,BLD2019-01033,8614,None,MICHIGAN,None,"Remodeling Inside shop, to be grocery store",CofO Issued,2019-07-02,20004672.,-83.147938,42.330158,590,POINT (-83.14794 42.33016)
423,BLD2020-00045,7311,None,MICHIGAN,None,"interior alterations, as per documents.",CofO Issued,2021-08-11,18002354.,-83.133627,42.330849,424,POINT (-83.13363 42.33085)
132,BLD2019-00831,2221,None,TRUMBULL,None,AKA 2221 Trumbull (Unit 6). Construction of a ...,CofO Issued,2020-06-03,08000587-9,-83.068029,42.332949,133,POINT (-83.06803 42.33295)
562,BLD2019-00068,500,None,Griswold,None,Change of Use. Interior Alterations for Carry-...,CofO Issued,2019-07-18,02001990-2,-83.046104,42.329581,563,POINT (-83.04610 42.32958)
133,BLD2020-00630,820,None,BALTIMORE,None,Revise BLD2019-04476 to modify code of review...,CofO Issued,2021-12-15,04001379.,-83.078371,42.366286,134,POINT (-83.07837 42.36629)
304,BLD2020-01793,13741,None,8 MILE,None,Change of Occupancy to Retail (Tobacco Special...,CofO Issued,2020-10-28,22018597.,-83.183087,42.444865,305,POINT (-83.18309 42.44487)
469,BLD2019-01064,2225,None,Trumbull,None,"(a.k.a. 2233 Trumbull, Unit 9 - local address)...",CofO Issued,2020-12-02,08000587-9,-83.068029,42.332949,470,POINT (-83.06803 42.33295)
479,BLD2019-01195,19158,None,LIVERNOIS,None,(AKA 19148 Livernois ) Change of Use/ Occupanc...,CofO Issued,2020-09-02,02004411-2,-83.141168,42.432492,480,POINT (-83.14117 42.43249)
177,BLD2019-00479,9212,None,DEXTER,None,CHANGE OF USE FROM RETAIL TO PRIVATE CLUB PER ...,CofO Issued,2019-05-09,12010456-9,-83.117443,42.370669,178,POINT (-83.11744 42.37067)


In [73]:
occupancy_gdf_empty[occupancy_gdf_empty['record_id']=='BLD2019-00680']

,record_id,street_num,street_dir,street_nam,street_typ,descriptio,status,date_statu,parcel_id,lon,lat,ObjectId,geometry
84,BLD2019-00680,2809,None,Brush,None,"Erect 4 story , 8 unit townhomes as per eplan...",CofO Issued,2021-04-05,None,NaN,NaN,85,None


In [74]:
occupancy_gdf_empty['descriptio'][occupancy_gdf_empty['record_id']=='BLD2019-00680'].values

array(['Erect  4 story , 8 unit townhomes as per eplans w/ a certificate of appropriateness'],
      dtype=object)

In [37]:
occupancy_gdf_empty['descriptio'][occupancy_gdf_empty['record_id']=='BLD2020-01564'].values

array(['Modify previous Change of Use Permit to Provisioning Center by adding grow facility; changes to the restroom facilities.'],
      dtype=object)

In [38]:
occupancy_gdf_empty['descriptio'][occupancy_gdf_empty['record_id']=='BLD2019-00033'].values

array(['INTERIOR ALTERATIONS TO ESTABLISH USE FOR TENANT SPACE AS COSMETIC RETAIL\nPERMANENT CERTIFICATE OF OCCUPANCY ISSUED (03-20-2019)'],
      dtype=object)

In [64]:
occupancy_gdf_empty['descriptio'][occupancy_gdf_empty['record_id']=='BLD2020-04413'].values[0]

'Interior alterations per plans.(1500 E. Woodbridge Suite address per plans, Separate Tenant Build-Out Permit required to establish Occupancy). Subject to all Applicable Federal, State, and Local Executive Orders.\r\n(AKA 1583 Franklin)'

- note: this building is at the corner of e. woodbridge and franklin (hence the aka 1583 franklin)

In [65]:
occupancy_gdf_empty[occupancy_gdf_empty['record_id']=='BLD2020-04413']

,record_id,street_num,street_dir,street_nam,street_typ,descriptio,status,date_statu,parcel_id,lon,lat,ObjectId,geometry
393,BLD2020-04413,1522,None,WOODBRIDGE,None,Interior alterations per plans.(1500 E. Woodbr...,CofO Issued,2021-04-12,None,NaN,NaN,394,None


In [41]:
occupancy_gdf_empty['descriptio'][occupancy_gdf_empty['record_id']=='BLD2019-04976'].values

array(["AKA 2327 Trumbull Ave. Unit 20. Per BZA #4-18, Construct 34' L X 21' W X 37' H Townhouse per plans."],
      dtype=object)

In [43]:
occupancy_gdf_empty['descriptio'][occupancy_gdf_empty['record_id']=='BLD2017-06240'].values

array(['AKA 8032, 8040, 8046, 8056 MEMORIAL. ERECTION OF ONE 4 UNIT ONE STORY WOOD FRAMED TOWNHOUSE AS PER PLANS. SEE BLD2017-00831 FOR MASTER SET OF PLANS.'],
      dtype=object)

In [9]:
rental_gdf = gpd.read_file('../data/Rental_Statuses/Rental_Statuses.shp')

In [18]:
len(rental_gdf[rental_gdf['geometry'].isna()])/len(rental_gdf)

0.00803870216790339

In [19]:
len(rental_gdf[rental_gdf['geometry'].isna()])

221

In [20]:
len(rental_gdf)

27492

## Geocoding
- first 2.5k free...?
https://developers.google.com/maps/documentation/geocoding/#Limits